<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-14 14:28:51
-------------------
qualified stocks: 93
with latest results: 28
still star stocks: 18
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  2.51 L
Current:  1.33 C
-------------------
Today PnL: -24.57 K (-0.18%)
Current PnL: -22.06 L (-15.01%)
CY Booked + Current PnL: -13.79 L (-9.38%)
-------------------
Total profit:  2.60 L
Total loss:  -24.66 L
-------------------
Total Booked + Current PnL: 13.19 L (10.99%)
Total Booked PnL: 35.25 L (29.36%)
Curr Year Booked PnL: 8.27 L (6.23%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.14 C
Est FTT PnL: 81.54 L (61.39%)
Deployed:  1.20 C
Current:  1.33 C
CAGR/XIRR %: 6.48%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SBIN,760.30,863.00,-11.34,M-LC,3.82,218824.0,17344.0,9869.0,0.47,8.61,4.51,13.51,61.0,1.76,1.68,22.29,XY25,NTT,BANKS
36,HINDALCO,651.95,756.01,-19.90,H-LC,6.21,110856.0,6544.0,10110.0,-1.09,6.27,9.12,15.96,11.0,0.65,0.85,24.23,X5K,ATH,METALS
4,APOLLOHOSP,7098.95,8285.00,-13.97,H-LC,5.93,171952.0,15775.0,10317.0,0.10,10.10,6.00,16.71,19.0,1.53,1.32,29.13,X40N,BTT,HEALTHCARE
55,MEDANTA,1087.93,1486.00,11.34,H-MC,13.82,158920.0,32720.0,13461.0,1.06,25.93,8.47,36.59,111.0,2.43,1.22,41.69,XY24,NTT,HEALTHCARE
51,LALPATHLAB,2838.53,3545.00,-23.88,H-SC,10.25,227270.0,34250.0,13795.0,1.42,17.74,6.07,24.89,134.0,2.48,1.74,45.23,X40N,NTT,HEALTHCARE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,RAJOOENG,102.50,143.33,-32.60,H-SC,4.60,102150.0,-350.0,41177.0,1.14,-0.34,40.31,39.83,116.0,-0.01,0.78,1.14,AR,ATH,MISC
25,DABUR,505.20,735.00,-10.32,H-MC,3.46,195156.0,-1872.0,91489.0,-0.62,-0.95,46.88,45.49,103.0,-0.02,1.50,11.97,XY24,BTT,FMCG
90,VOLTAS,1278.28,1918.49,-9.05,H-MC,7.73,190740.0,-1002.0,97029.0,-0.35,-0.52,50.87,50.08,101.0,-0.01,1.46,6.24,XY25,ATH,AC
58,PGHH,13388.00,18062.58,-31.08,H-MC,6.35,199500.0,-1320.0,71441.0,1.53,-0.66,35.81,34.92,82.0,-0.02,1.53,3.99,X40,ATH,FMCG


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,CIPLA,1492.70,1795.0,-19.78,H-LC,2.52,214158.0,9658.0,31760.0,0.14,4.72,14.83,20.25,10.0,0.30,1.64,13.87,X40N,BTT,PHARMA
5,ASIANPAINT,2961.56,4250.0,-11.88,H-LC,3.20,171768.0,-29618.0,117232.0,1.03,-14.71,68.25,43.51,27.0,-0.25,1.32,19.57,X40,BTT,PAINTS
57,NESTLEIND,2268.60,2755.0,-17.39,H-LC,3.24,127085.0,-138341.0,195253.0,-0.98,-52.12,153.64,21.44,8.0,-0.71,0.98,2.12,XY25,NTT,FMCG
53,LTIM,5564.16,7230.2,-6.94,H-LC,3.75,184680.0,-15630.0,75608.0,0.30,-7.80,40.94,29.94,17.0,-0.21,1.42,27.59,X200,ATH,IT
37,HINDUNILVR,2413.81,2723.0,-11.81,H-LC,5.76,277200.0,6853.0,27775.0,-0.81,2.53,10.02,12.81,28.0,0.25,2.13,15.93,XY25,NTT,FMCG


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
88,VALIANTORG,512.64,838.0,-325.20,H-SC,16.79,95823.0,-37463.0,122059.0,5.00,-28.11,127.38,63.47,143.0,-0.31,0.74,60.62,XR,NTT,CHEMICALS
10,BAJAJHFL,122.26,152.0,-12.15,H-MC,5.31,185320.0,-15186.0,63954.0,4.09,-7.57,34.51,24.33,98.0,-0.24,1.42,6.87,X40N,BTT,FINANCE
29,EASEMYTRIP,18.12,26.4,-7.85,M-SC,24.82,88700.0,-88115.0,168911.0,2.60,-49.83,190.43,45.70,197.0,-0.52,0.68,3.18,XY24,NTT,TRAVEL
92,WIPRO,243.46,420.0,-12.89,M-LC,7.31,153407.0,2462.0,106986.0,2.39,1.63,69.74,72.51,54.0,0.02,1.18,7.75,XR,NTT,IT
30,GREENPANEL,375.16,537.0,200.39,M-SC,5.70,134127.0,-48951.0,127930.0,2.35,-26.74,95.38,43.14,230.0,-0.38,1.03,26.16,XY24,NTT,MISC


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BATAINDIA,1550.24,2096.0,-42.29,M-SC,7.22,86378.0,-42292.0,87587.0,-5.12,-32.87,101.40,35.20,209.0,-0.48,0.66,0.00,X40,NTT,FOOTWEAR
65,SAIL,130.64,228.0,31.52,M-MC,10.18,207484.0,-17478.0,185138.0,-2.87,-7.77,89.23,74.53,193.0,-0.09,1.59,20.47,XY24,BTT,STEEL
84,UJJIVANSFB,52.77,60.0,77.36,H-SC,20.39,112320.0,-30159.0,49679.0,-2.62,-21.17,44.23,13.70,163.0,-0.61,0.86,32.57,OX40N,NTT,BANKS
70,SHALBY,261.39,327.0,935.09,M-SC,6.89,140811.0,-40332.0,85796.0,-2.49,-22.27,60.93,25.10,232.0,-0.47,1.08,12.85,XY24,NTT,HEALTHCARE
72,SIS,477.00,528.0,2111.40,H-SC,6.37,59500.0,-21590.0,30262.0,-2.10,-26.62,50.86,10.69,155.0,-0.71,0.46,20.59,OX40N,NTT,MISC


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TTKPRESTIG,769.29,770.0,92.96,M-SC,10.34,83126.0,-17651.0,17747.0,-0.08,-17.51,21.35,0.09,243.0,-0.99,0.64,9.21,OX40N,NTT,DURABLES
20,CERA,7989.07,9475.0,-19.57,H-SC,3.11,114786.0,-29017.0,55763.0,-0.52,-20.18,48.58,18.60,148.0,-0.52,0.88,26.67,OX40N,NTT,CERAMICS
84,UJJIVANSFB,52.77,60.0,77.36,H-SC,20.39,112320.0,-30159.0,49679.0,-2.62,-21.17,44.23,13.70,163.0,-0.61,0.86,32.57,OX40N,NTT,BANKS
43,INDIGOPNTS,1407.73,1408.0,109.26,M-SC,2.91,137392.0,-37167.0,37206.0,0.80,-21.29,27.08,0.02,221.0,-1.00,1.05,19.14,OX40N,NTT,PAINTS
49,KANSAINER,299.63,340.0,-69.25,H-SC,2.82,212256.0,-57411.0,93753.0,-0.27,-21.29,44.17,13.47,142.0,-0.61,1.63,7.36,XY24,NTT,PAINTS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,TCS,3794.03,4998.00,-28.69,H-LC,15.98,235708.0,-60226.0,154129.0,-0.47,-20.35,65.39,31.73,2.0,-0.39,1.81,0.63,X40,BTT,IT
45,INFY,1461.46,2275.00,-20.67,H-LC,10.92,259729.0,-1872.0,147500.0,1.71,-0.72,56.79,55.67,4.0,-0.01,1.99,5.35,X40,BTT,IT
89,VBL,492.64,671.64,-11.16,H-LC,10.14,269834.0,4794.0,91501.0,-1.87,1.81,33.91,36.33,5.0,0.05,2.07,15.35,X40N,ATH,FMCG
1,ABB,5551.76,7934.00,-42.20,H-LC,10.47,196228.0,-20291.0,113204.0,-1.09,-9.37,57.69,42.91,7.0,-0.18,1.51,2.77,AR,NTT,ELECTRICAL
57,NESTLEIND,2268.60,2755.00,-17.39,H-LC,3.24,127085.0,-138341.0,195253.0,-0.98,-52.12,153.64,21.44,8.0,-0.71,0.98,2.12,XY25,NTT,FMCG


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.0,11.34,H-MC,13.82,158920.0,32720.0,13461.0,1.06,25.93,8.47,36.59,111.0,2.43,1.22,41.69,XY24,NTT,HEALTHCARE


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.00,11.34,H-MC,13.82,158920.0,32720.0,13461.0,1.06,25.93,8.47,36.59,111.0,2.43,1.22,41.69,XY24,NTT,HEALTHCARE
50,KPIGREEN,497.21,731.35,7.49,H-SC,12.20,128205.0,2908.0,56090.0,-1.30,2.32,43.75,47.09,141.0,0.05,0.98,60.46,MH,ATH,POWER
15,BLUESTARCO,1646.70,2326.38,3.35,H-MC,6.93,178500.0,13830.0,54139.0,1.66,8.40,30.33,41.28,91.0,0.26,1.37,17.05,X40N,ATH,AC
19,CAMS,3643.00,5211.76,-4.72,H-SC,3.88,108248.0,6244.0,37681.0,1.17,6.12,34.81,43.06,123.0,0.17,0.83,24.29,X40N,ATH,MISC
35,HEROMOTOCO,4311.81,5949.07,-11.08,H-MC,7.82,165095.0,14182.0,43123.0,-1.08,9.40,26.12,37.97,92.0,0.33,1.27,37.08,AR,ATH,AUTO


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,-11.08,H-MC,7.82,165095.0,14182.0,43123.0,-1.08,9.40,26.12,37.97,92.0,0.33,1.27,37.08,AR,ATH,AUTO
42,INDIAMART,2327.09,4850.92,-51.65,H-SC,11.98,134864.0,11528.0,122241.0,-0.32,9.35,90.64,108.45,119.0,0.09,1.04,33.79,AR,ATH,MISC
50,KPIGREEN,497.21,731.35,7.49,H-SC,12.20,128205.0,2908.0,56090.0,-1.30,2.32,43.75,47.09,141.0,0.05,0.98,60.46,MH,ATH,POWER
92,WIPRO,243.46,420.00,-12.89,M-LC,7.31,153407.0,2462.0,106986.0,2.39,1.63,69.74,72.51,54.0,0.02,1.18,7.75,XR,NTT,IT
91,WHIRLPOOL,1219.98,2270.00,-45.21,M-SC,4.29,92708.0,1210.0,77541.0,-0.42,1.32,83.64,86.07,219.0,0.02,0.71,34.11,XR,NTT,DURABLES


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-49.70,L-SC,21.61,73152.0,-40397.0,80445.0,0.00,-35.58,109.97,35.27,268.0,-0.50,0.56,79.04,XR,NTT,HOTELS
3,ALKYLAMINE,2347.98,4546.37,-4.00,H-SC,15.59,89358.0,-11605.0,106139.0,-0.71,-11.49,118.78,93.63,149.0,-0.11,0.69,33.10,SR,ATH,CHEMICALS
54,MASFIN,326.60,399.50,-20.05,H-SC,13.84,90990.0,-6990.0,28862.0,-1.45,-7.13,31.72,22.32,152.0,-0.24,0.70,31.97,XR,ATH,FINANCE
88,VALIANTORG,512.64,838.00,-325.20,H-SC,16.79,95823.0,-37463.0,122059.0,5.00,-28.11,127.38,63.47,143.0,-0.31,0.74,60.62,XR,NTT,CHEMICALS
36,HINDALCO,651.95,756.01,-19.90,H-LC,6.21,110856.0,6544.0,10110.0,-1.09,6.27,9.12,15.96,11.0,0.65,0.85,24.23,X5K,ATH,METALS


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LALPATHLAB,2838.53,3545.00,-23.88,H-SC,10.25,227270.0,34250.0,13795.0,1.42,17.74,6.07,24.89,134.0,2.48,1.74,45.23,X40N,NTT,HEALTHCARE
35,HEROMOTOCO,4311.81,5949.07,-11.08,H-MC,7.82,165095.0,14182.0,43123.0,-1.08,9.40,26.12,37.97,92.0,0.33,1.27,37.08,AR,ATH,AUTO
23,COFFEEDAY,59.14,80.00,-49.70,L-SC,21.61,73152.0,-40397.0,80445.0,0.00,-35.58,109.97,35.27,268.0,-0.50,0.56,79.04,XR,NTT,HOTELS
70,SHALBY,261.39,327.00,935.09,M-SC,6.89,140811.0,-40332.0,85796.0,-2.49,-22.27,60.93,25.10,232.0,-0.47,1.08,12.85,XY24,NTT,HEALTHCARE
28,DMART,3764.92,5201.00,-14.05,H-LC,6.55,214630.0,30149.0,40222.0,1.05,16.34,18.74,38.14,35.0,0.75,1.65,28.68,X40N,NTT,FMCG


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.45
1,25,41.16
2,50,71.01


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    39.34
LC    34.38
MC    26.31
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.80
X40N     15.65
X40      13.38
XY25     10.65
XR        9.09
AR        8.43
OX40N     7.47
X5K       2.27
MH        1.74
X200      1.42
SR        1.13
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    26.00
H-SC    24.53
H-MC    23.38
M-SC    13.32
M-LC     7.29
M-MC     2.59
L-SC     1.49
L-LC     1.09
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.20,-11.55,47.41
IT,12.03,-19.12,78.61
FINANCE,8.27,-21.04,65.82
BANKS,7.61,-15.06,63.91
MISC,5.92,-19.27,87.65
PAINTS,5.66,-18.39,41.72
HEALTHCARE,5.36,6.07,17.65
ELECTRICAL,4.74,-13.81,63.10
AUTO,4.48,-15.67,65.99


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2883471.0,23
XR,1111748.0,13
AR,1111675.0,10
X40,828889.0,10
XY25,662586.0,8
X40N,567574.0,12
OX40N,484293.0,10
SR,196531.0,2
X5K,143058.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2821263.0,28
M-SC,1509528.0,17
H-LC,1352419.0,18
H-MC,1331948.0,18
M-LC,428696.0,5
M-MC,339932.0,2
L-SC,267083.0,3
L-MC,58678.0,1
L-LC,44225.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      947973.0      6
M-SC       XY24      875723.0      7
H-SC       AR        674345.0      5
           XR        524150.0      6
H-LC       X40       514464.0      5
H-MC       XY24      499645.0      5
M-MC       XY24      339932.0      2
H-LC       XY25      261125.0      3
           AR        233064.0      2
H-MC       X40       226838.0      4
           XY25      222864.0      2
H-LC       X40N      216743.0      5
H-SC       OX40N     211540.0      4
           X40N      210634.0      4
           SR        196531.0      2
M-LC       XY24      178893.0      2
L-SC       XR        176559.0      2
M-SC       AR        161143.0      2
H-MC       X40N      140197.0      3
M-SC       OX40N     133457.0      4
M-LC       X5K       132948.0      1
M-SC       XR        127115.0      2
           XY25      124503.0      1
H-MC       XR        118260.0      1
M-LC       XR        106986.0      1
L-SC       OX40N      90524.0      1
M-SC       X40        87587.0      1
H-LC       X200       75608.0      1
L-MC       XR         58678.0      1
H-SC       MH         56090.0      1
H-MC       OX40N      48772.0      1
L-LC       XY25       44225.0      1
H-MC       AR         43123.0      1
H-LC       XY24       41305.0      1
H-MC       MH         32249.0      1
H-LC       X5K        10110.0      1
M-LC       XY25        9869.0      1

In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
